# Chinese exports by year, country and commodity

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import glob

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
# pd.set_option("display.float_format", lambda x: "%.2f" % x)
pd.options.display.float_format = "${:,.2f}".format

---

### Read processed trade data

In [4]:
df = pd.read_csv(
    "../data/processed/exports_xingjiang_all_countries.csv",
    dtype={
        "commodity_code": str,
        "commodity_chapter": str,
        "commodity_chapter_four": str,
    },
)

In [5]:
df = df.drop_duplicates()

---

### Exports to countries

In [6]:
xinjiang_exports = df.groupby(["exported_to"]).agg({"us_dollar": "sum"}).reset_index()

In [7]:
df[df["exported_to"] == "Kazakhstan"].to_csv(
    "../data/processed/kazakhstan.csv", index=False
)

In [8]:
xinjiang_exports.sort_values("us_dollar", ascending=False).head(20)

,exported_to,us_dollar
84,Kazakhstan,23567538140
87,Kyrgyzstan,14866972113
140,Russia,4339204979
165,Tajikistan,2919888343
181,Uzbekistan,1555308229
125,Pakistan,1103146920
179,United States,1070939568
74,India,1017187461
184,Vietnam,767979276
80,Italy,456837042


In [9]:
xinjiang_exports.to_csv(
    "../data/processed/xinjiang_exports_countries_2017_2021.csv", index=False
)

In [10]:
df.to_csv("../data/processed/xinjiang_all_countries_exports.csv", index=False)

---

### Most exported commodity types

In [11]:
df[
    [
        "commodity_chapter_desc_clean",
        "commodity_chapter_desc_clean_two",
        "trash",
        "trash",
        "trash",
        "trash",
    ]
] = df["commodity_chapter_desc"].str.split(pat=";", expand=True)

In [12]:
xinjiang_exports_types = (
    df.groupby(["commodity_chapter_desc_clean"]).agg({"us_dollar": sum}).reset_index()
)

In [13]:
xinjiang_exports_types.sort_values("us_dollar", ascending=False).head(10)

,commodity_chapter_desc_clean,us_dollar
5,Apparel and clothing accessories,19506957324
28,Footwear,8530093335
60,Plastics and articles thereof,4450636812
7,Articles of leather,2424928662
56,Organic chemicals,2136200606
20,Electrical machinery and equipment and parts thereof,2125848899
62,"Preparations of vegetables, fruit, nuts or other parts of plants",2012005412
29,"Fruit and nuts, edible",1510275242
77,"Textiles, made up articles",1494637674
36,Iron or steel articles,1378102247


In [14]:
xinjiang_exports_types.sort_values("us_dollar", ascending=False).head(10).to_csv(
    "../data/processed/xinjiang_exports_commodities_2017_2021.csv", index=False
)

### Exports by year

In [15]:
xinjiang_exports_types_year = (
    df.groupby(["year", "commodity_chapter_desc_clean"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [16]:
xinjiang_exports_types_year_pivot = (
    xinjiang_exports_types_year.pivot_table(
        columns="year", index="commodity_chapter_desc_clean", values="us_dollar"
    )
    .fillna(0)
    .reset_index()
)

In [17]:
xinjiang_exports_types_year_pivot.columns = (
    xinjiang_exports_types_year_pivot.columns.astype(str)
)

In [18]:
xinjiang_exports_types_year_pivot["2021"] = xinjiang_exports_types_year_pivot[
    "2021"
].round(0)

In [19]:
xinjiang_exports_types_year_pivot.head()

year,commodity_chapter_desc_clean,2017,2018,2019,2020,2021
0,"Aircraft, spacecraft and parts thereof",$0.00,$0.00,$0.00,$0.00,"$58,259.00"
1,Albuminoidal substances,"$9,823,866.00","$11,648,087.00","$11,273,195.00","$7,815,793.00","$8,122,353.00"
2,Aluminium and articles thereof,"$64,529,277.00","$116,933,622.00",$0.00,$0.00,"$112,382,667.00"
3,Animal or vegetable fats and oils and their cleavage products,"$126,051.00","$104,069.00","$82,822.00","$48,143.00","$56,431.00"
4,Animal originated products,"$17,981,235.00","$17,393,941.00","$10,039,977.00","$8,030,283.00","$10,654,213.00"


In [20]:
xinjiang_exports_types_year_pivot.sort_values("2021", ascending=False).head(10)

year,commodity_chapter_desc_clean,2017,2018,2019,2020,2021
5,Apparel and clothing accessories,"$4,742,867,777.00","$4,005,682,817.00","$4,565,794,735.00","$187,337,449.00","$6,005,274,546.00"
20,Electrical machinery and equipment and parts thereof,$0.00,$0.00,$0.00,$0.00,"$2,125,848,899.00"
28,Footwear,"$2,890,864,457.00","$1,958,535,325.00","$1,865,182,639.00",$0.00,"$1,815,510,914.00"
60,Plastics and articles thereof,"$720,671,878.00","$747,512,043.00","$760,309,024.00","$863,236,970.00","$1,358,906,897.00"
52,"Nuclear reactors, boilers, machinery and mechanical appliances",$0.00,$0.00,$0.00,$0.00,"$1,016,381,537.00"
87,Vehicles,$0.00,$0.00,$0.00,$0.00,"$782,275,096.00"
81,"Toys, games and sports requisites",$0.00,$0.00,$0.00,$0.00,"$767,135,447.00"
30,Furniture,$0.00,$0.00,$0.00,$0.00,"$642,189,079.00"
56,Organic chemicals,"$268,769,145.00","$397,794,487.00","$479,526,675.00","$384,541,305.00","$605,568,994.00"
36,Iron or steel articles,"$350,163,622.00","$466,211,827.00","$132,305,002.00",$0.00,"$429,421,796.00"


In [21]:
xinjiang_exports_types_year_pivot["2017-2020-mean"] = xinjiang_exports_types_year_pivot[
    ["2017", "2018", "2019", "2020"]
].mean(axis=1)

In [22]:
# xinjiang_exports_types_year_pivot["change"] = round(
#     (
#         xinjiang_exports_types_year_pivot["2021"]
#         - xinjiang_exports_types_year_pivot["2017-2020-mean"]
#     )
#     / xinjiang_exports_types_year_pivot["2017-2020-mean"]
#     * 100
# )

In [23]:
xinjiang_exports_types_year_pivot["2021_diff_from_mean"] = (
    xinjiang_exports_types_year_pivot["2021"]
    - xinjiang_exports_types_year_pivot["2017-2020-mean"]
)

In [24]:
xinjiang_exports_types_year_pivot[
    xinjiang_exports_types_year_pivot["2017-2020-mean"] > 0
].sort_values("2021", ascending=False).head(20)

year,commodity_chapter_desc_clean,2017,2018,2019,2020,2021,2017-2020-mean,2021_diff_from_mean
5,Apparel and clothing accessories,"$4,742,867,777.00","$4,005,682,817.00","$4,565,794,735.00","$187,337,449.00","$6,005,274,546.00","$3,375,420,694.50","$2,629,853,851.50"
28,Footwear,"$2,890,864,457.00","$1,958,535,325.00","$1,865,182,639.00",$0.00,"$1,815,510,914.00","$1,678,645,605.25","$136,865,308.75"
60,Plastics and articles thereof,"$720,671,878.00","$747,512,043.00","$760,309,024.00","$863,236,970.00","$1,358,906,897.00","$772,932,478.75","$585,974,418.25"
56,Organic chemicals,"$268,769,145.00","$397,794,487.00","$479,526,675.00","$384,541,305.00","$605,568,994.00","$382,657,903.00","$222,911,091.00"
36,Iron or steel articles,"$350,163,622.00","$466,211,827.00","$132,305,002.00",$0.00,"$429,421,796.00","$237,170,112.75","$192,251,683.25"
7,Articles of leather,"$593,675,210.00","$542,374,254.00","$481,571,958.00","$389,440,790.00","$417,866,450.00","$501,765,553.00","$-83,899,103.00"
35,Iron and steel,"$301,944,321.00","$281,736,935.00","$301,229,891.00",$0.00,"$365,289,725.00","$221,227,786.75","$144,061,938.25"
62,"Preparations of vegetables, fruit, nuts or other parts of plants","$390,828,456.00","$412,565,467.00","$475,350,587.00","$389,791,795.00","$343,469,107.00","$417,134,076.25","$-73,664,969.25"
29,"Fruit and nuts, edible","$306,523,578.00","$291,505,857.00","$391,548,409.00","$278,683,865.00","$242,013,533.00","$317,065,427.25","$-75,051,894.25"
10,Ceramic products,"$151,710,927.00","$263,094,326.00","$398,134,995.00",$0.00,"$241,343,729.00","$203,235,062.00","$38,108,667.00"


### Exports by year

In [25]:
agg_exports_by_year = df.groupby(["year"]).agg({"us_dollar": "sum"}).reset_index()

In [26]:
agg_exports_by_year.us_dollar.pct_change() * 100

0       NaN
1   $-12.39
2     $1.94
3   $-69.33
4   $435.32
Name: us_dollar, dtype: float64

In [27]:
agg_exports_by_year.head()

,year,us_dollar
0,2017,13436995662
1,2018,11771866361
2,2019,12000016602
3,2020,3680144438
4,2021,19700596110


In [28]:
agg_exports_by_year.to_csv(
    "../data/processed/xinjiang_exports_agg_years_2017_2021.csv", index=False
)

---

### Cotton exports

In [29]:
xinjiang_52_exports = (
    df[df["commodity_chapter"] == "52"]
    .groupby(["commodity_chapter_desc", "exported_to"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [30]:
xinjiang_52_exports.sort_values("us_dollar", ascending=False).head(20)

,commodity_chapter_desc,exported_to,us_dollar
26,Cotton,Kyrgyzstan,99004238
25,Cotton,Kazakhstan,61101229
66,Cotton,Vietnam,29191946
39,Cotton,Pakistan,22047789
59,Cotton,Tajikistan,16899388
24,Cotton,Italy,6559469
65,Cotton,Uzbekistan,5733623
4,Cotton,Bangladesh,5026244
62,Cotton,United Arab Emirates,3830263
56,Cotton,Spain,2470699


### Tomatoes

In [31]:
xinjiang_0702_exports = (
    df[df["commodity_chapter_four"] == "0702"]
    .groupby(["year", "commodity_chapter_four_desc", "exported_to"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [32]:
xinjiang_0702_exports.sort_values("us_dollar", ascending=False)

,year,commodity_chapter_four_desc,exported_to,us_dollar
0,2017,Tomatoes; fresh or chilled,Kazakhstan,7889295
4,2018,Tomatoes; fresh or chilled,Kazakhstan,7721800
3,2017,Tomatoes; fresh or chilled,Russia,5449900
7,2019,Tomatoes; fresh or chilled,Kazakhstan,5327499
1,2017,Tomatoes; fresh or chilled,Kyrgyzstan,2033227
9,2019,Tomatoes; fresh or chilled,Russia,1678446
6,2018,Tomatoes; fresh or chilled,Russia,1530022
8,2019,Tomatoes; fresh or chilled,Kyrgyzstan,753862
10,2020,Tomatoes; fresh or chilled,Kazakhstan,683078
15,2021,Tomatoes; fresh or chilled,Kyrgyzstan,662146


In [33]:
xinjiang_0702_exports.pivot_table(
    index="commodity_chapter_four_desc", values="us_dollar", columns="year"
).reset_index()

year,commodity_chapter_four_desc,2017,2018,2019,2020,2021
0,Tomatoes; fresh or chilled,"$3,844,136.50","$3,181,994.00","$2,586,602.33","$254,443.50","$252,970.00"


### Polysilicon

In [34]:
xinjiang_280461_exports = (
    df[df["commodity_code"].str.contains("280461")]
    .groupby(["year", "commodity_chapter_four_desc", "exported_to"])
    .agg({"us_dollar": sum})
    .reset_index()
)

In [35]:
xinjiang_280461_exports

,year,commodity_chapter_four_desc,exported_to,us_dollar
0,2017,"Hydrogen, rare gases and other non-metals",Pakistan,5893160
1,2019,"Hydrogen, rare gases and other non-metals",Italy,4854
2,2020,"Hydrogen, rare gases and other non-metals",Belgium,494
3,2020,"Hydrogen, rare gases and other non-metals",Croatia,1
4,2020,"Hydrogen, rare gases and other non-metals",Reunion,2
